## **Questão 1**

#### **Explicando alguns termos**


##### **1) Backtest**

Backtest é o processo de testar uma estrátegia de investimentos ou modelo de trading usando dados históricos. O objetivo de um backtest é
avaliar como a estratégia elaborada teria se comportado no passado para, a partir disso, inferir como pode ser o potencial futuro dessa
estratégia.

##### **2) Dividendos**

Dividendos são pagamentos distribuidos por uma certa empresa a seus acionistas como uma parcela do lucro da empresa. Eles são uma forma de retorno de investimento para os acionistas e podem ser pagos em dinheiro ou em ações adicionais.

##### **3) Split**

Um split, ou split de ações, é um evento corporativo onde uma certa empresa decide dividir suas ações existentes em um número maior de ações. Isso geralmente é feito para tornar o preço dessas ações mais acessível aos investidores, sem alterar o valor total da empresa.

##### **4) Reverse split**

Um reverse split é um evento corporativo em que uma empresa reduz o número de suas ações em circulação ao consolidar várias ações existentes em uma única ação nova. Isso acaba resultando em um aumento no preço das ações de tal empresa, sem alterar o valor total de mercado da mesma.

##### **5) Spin-off**

Um spin-off é um evento corporativo em que uma empresa cria uma nova empresa independente, separando uma parte de seus negócios. Em tal evento, os acionistas da empresa original geralmente recebem ações da nova empresa, proporcionais a sua participação na empresa original.

##### **6) Adjustment close**

O adjustment close é basicamente o preço de uma ação ajustado a ocorrência de eventos corporativos como dividendos, splits, reverse splits e spin-offs. Esse tipo de ajuste é feito para fornecer uma visão mais precisa do desempenho de uma ação ao longo do tempo, levando em conta as mudanças que afetam o preço.

#### **Resolução da questão**

In [3]:
# Instalando a biblioteca do Yahoo Finance
%pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Importando as bibliotecas que serão utilizadas
import yfinance as yf

In [9]:
# Questão 1 - a) Importando dados da Vale (VALE3).

# Salvo em uma variável o ticker (símbolo de identificação) da ação desejada.
ticker = "VALE3.SA"

# Obtenho os dados da ação desejada
data = yf.download(ticker, "2018-01-01", "2022-12-31")

# Exibo os dados obtidos da API do Yahoo Finance (Os dados são retornados como um dataframe do Pandas).
print(data)


[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2018-01-02  40.439999  41.740002  40.439999  41.720001  26.179432  14156500
2018-01-03  41.830002  41.880001  41.299999  41.470001  26.022564  12744200
2018-01-04  41.810001  42.369999  41.520000  41.639999  26.129234  18433000
2018-01-05  41.570000  42.290001  41.310001  42.290001  26.537115  15251300
2018-01-08  42.400002  43.230000  42.400002  43.230000  27.126965  14542800
...               ...        ...        ...        ...        ...       ...
2022-12-23  85.279999  86.519997  85.080002  86.320000  77.709412  18656500
2022-12-26  86.419998  87.540001  86.330002  87.110001  78.420601  18407500
2022-12-27  88.000000  89.650002  87.470001  89.190002  80.293129  26720300
2022-12-28  89.190002  90.209999  88.699997  88.989998  80.113068  23314500
2022-12-29  89.000000  89.949997  88.830002  88.879997  80.014038  40110800

[1239 rows 